## Plot markers umap embeddings
Does this over each donor and separate

In [1]:
# Input info
se_f = "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/gff_A2_black/annotation_clones/SE.rds"
#sample_names = "preB,postB"
outdir = "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap"


# Parallel processing
cores = 4

# R utils script
utils_script = "/data/Mito_Trace/R_scripts/seurat_utils.R"
markers_f = "/data/Mito_Trace/data/processed/cd34_markers_bc.txt"



In [2]:
library("Nebulosa")

Loading required package: ggplot2

Warning message:
“package ‘ggplot2’ was built under R version 4.1.3”
Loading required package: patchwork

Registered S3 method overwritten by 'spatstat.core':
  method          from
  formula.glmmPQL MASS



In [3]:
source(utils_script)
#outdir = outdir
dir.create(outdir, showWarnings = FALSE)


Attaching SeuratObject

Attaching sp

Warning message:
“package ‘dplyr’ was built under R version 4.1.3”

Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [4]:
# sample_names <- unlist(strsplit(sample_names, ",")[[1]])
# sample_names

In [5]:
library(GenomicRanges)
library(Seurat)
library(Signac)
library(GenomeInfoDb)
library(EnsDb.Hsapiens.v75)
library(ggplot2)
library(patchwork)
set.seed(1234)
library(data.table)
library(magrittr)
library(cowplot)
library(metap)
library("RColorBrewer")

library(future)
plan()

plan("multiprocess", workers = cores)
#options(future.globals.maxSize = 50000 * 1024^2) # for 50 Gb RAM
options(future.globals.maxSize = 8000 * 1024^2)

library(dplyr)

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors

Warning message:
“package ‘S4Vectors’ was built under R version 4.1.2”

Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:dplyr’:

    first, rename


The following objects are masked from 

sequential:
- args: function (..., envir = parent.frame())
- tweaked: FALSE
- call: NULL

Warning message:
"Strategy 'multiprocess' is deprecated in future (>= 1.20.0). Instead, explicitly specify either 'multisession' or 'multicore'. In the current R session, 'multiprocess' equals 'multicore'."


In [6]:
se <- readRDS(se_f)#file.path(indir, paste0("allSamples.integrated.rds")))
se

DefaultAssay(se) <- "RNA"

An object of class Seurat 
212927 features across 13593 samples within 2 assays 
Active assay: RNA (19039 features, 0 variable features)
 1 other assay present: ATAC
 2 dimensional reductions calculated: integrated_lsi, umap

In [7]:
markers <- read.csv(markers_f, header=F)[[1]]
markers <- unique(markers)

In [8]:
markers


[1] "LYZ"      "GAS7"     "MPO"      "ELANE"    "LPO"      "PRTN3"   
 [7] "IDS"      "GBP5"     "PTCHD1"   "YPEL5"    "CCNL1"    "VIM"     
[13] "ITM2C"    "ST3GAL6"  "PROC"     "HBB"      "ABCC4"    "HBG2"    
[19] "SOS1"     "THRB"     "CPOX"     "IL1RL1"   "CLC"      "IL18R1"  
[25] "IKZF2"    "IL5RA"    "MEIS1"    "CDK6"     "PTPN14"   "ANK1"    
[31] "LIMS1"    "HMBOX1"   "XXYLT1"   "DAPK1"    "PIP5K1B"  "DUSP22"  
[37] "CTSZ"     "GYPB"     "KCNK1"    "NCR2"     "TLR9"     "ICOS"    
[43] "CD81"     "D79A"     "CD79B"    "EBF1"     "GSDME"    "ARHGAP26"
[49] "RAB31"    "ATP8B4"   "FNDC3B"   "GCNT1"    "ETV6"     "NSMCE2"  
[55] "MED13L "

## Get the average expression in each group for each marker
## Get % expressed

In [13]:
avg_clust <- as.data.frame(AverageExpression(se, assays="RNA", group.by = "seurat_clusters",
                                verbose = FALSE)$RNA, stringsAsFactors=F, optional=F)[markers,]
avg_clust

exp <- FetchData(se, markers,)
exp

write.table(avg_clust, file=file.path(outdir, "avgExp.tsv"), quote=F, sep="\t")

# matrix <- as.matrix(colMeans(exp  > 0))*100
# matrix

exp <- merge(exp, se[["seurat_clusters"]], by="row.names")
write.table(exp, file=file.path(outdir, "markerExp.tsv"), quote=F, sep="\t")


,0,1,2,3,4,5,6,7,8,9,10,11
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
LYZ,0.050804444,0.037683908,0.029397995,0.014428173,0.125577176,0.041558455,0.024862246,0.023746881,0.04320287,0.062038981,0.018673862,0.021768929
GAS7,1.213701336,1.419354783,0.968074833,0.539559029,2.125760369,0.950985030,1.353854271,0.319717514,1.29990067,2.153089698,1.319260417,1.181125285
MPO,0.088601656,0.432147425,0.048079821,0.039040375,0.185073608,0.065609508,0.150710398,0.018651735,0.09757663,0.097910861,0.045386415,0.072222642
ELANE,0.017565251,0.157951613,0.006614465,0.027342026,0.062894373,0.017269426,0.013053111,0.006492386,0.11722573,0.025264767,0.008053197,0.006005432
LPO,0.176160370,0.667595216,0.162238876,0.244728928,0.370776161,0.341575342,0.226370290,0.157448950,0.31077685,0.335808653,0.147062511,0.184835308
PRTN3,0.019397721,0.147885597,0.010941616,0.026482887,0.060163489,0.034038214,0.019641548,0.014518900,0.07392181,0.025553456,0.006950422,0.023969309
IDS,0.453033470,0.241851770,0.777731988,0.206881318,0.278932476,0.388418942,0.507570827,0.388318698,0.28244349,0.304651661,0.256841997,0.513942016
GBP5,0.156114747,0.112901496,0.326185070,0.050330130,0.077183260,0.081645195,0.167095837,0.049657995,0.08427286,0.084185379,0.083242190,0.055959480
PTCHD1,0.128553715,0.061182926,0.300800666,0.084991482,0.044723640,0.153534254,0.128004343,0.259859524,0.10732529,0.051329355,0.070806238,0.055443585


Warning message in FetchData.Seurat(se, markers, ):
"The following requested variables were not found: D79A, MED13L "


,LYZ,GAS7,MPO,ELANE,LPO,PRTN3,IDS,GBP5,PTCHD1,YPEL5,⋯,CD79B,EBF1,GSDME,ARHGAP26,RAB31,ATP8B4,FNDC3B,GCNT1,ETV6,NSMCE2
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Flt3l_AAACGAAAGGAGACTC-1,0.000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0,0.0000000,⋯,0,0.0000000,0.0000000,1.8075560,0.0000000,1.8075560,0.0000000,0.0000000,1.807556,1.8075560
Flt3l_AAACGAAAGTCACGCC-1,0.000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0,0.0000000,⋯,0,1.0904147,0.0000000,1.0904147,0.0000000,0.0000000,0.0000000,0.0000000,1.599593,0.0000000
Flt3l_AAACGAACAAGACTAA-1,0.000000,0.6839030,0.0000000,0.0000000,1.0862677,0.0000000,0.0000000,0.0000000,0,0.0000000,⋯,0,0.0000000,0.6839030,1.5946060,0.0000000,0.6839030,0.6839030,0.0000000,1.086268,0.0000000
Flt3l_AAACGAACAAGCGAAC-1,0.000000,0.6852290,0.0000000,0.0000000,0.6852290,0.0000000,1.3743935,0.0000000,0,0.0000000,⋯,0,0.6852290,0.0000000,1.5967386,0.6852290,0.0000000,1.9322975,0.6852290,1.088041,0.0000000
Flt3l_AAACGAACACAGCCAC-1,0.000000,1.5160067,0.0000000,0.0000000,0.0000000,0.0000000,0.6357833,0.0000000,0,0.6357833,⋯,0,0.0000000,0.0000000,0.0000000,0.0000000,0.6357833,1.0213718,0.6357833,1.298965,1.2989654
Flt3l_AAACGAACACAGGTAG-1,0.000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0,0.0000000,⋯,0,0.0000000,0.0000000,0.0000000,0.0000000,1.5412375,0.0000000,0.0000000,0.000000,0.0000000
Flt3l_AAACGAACACAGTAGG-1,0.000000,0.6082509,0.9837364,0.0000000,0.0000000,0.0000000,0.6082509,0.0000000,0,0.6082509,⋯,0,0.0000000,0.6082509,1.6459773,0.9837364,1.2560848,1.2560848,0.6082509,1.925781,0.6082509
Flt3l_AAACGAACATAGAATG-1,1.394745,1.3947448,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0,0.0000000,⋯,0,0.0000000,0.0000000,0.0000000,1.3947448,1.3947448,0.0000000,1.3947448,1.394745,0.0000000
Flt3l_AAACGAACATGTAGAA-1,0.000000,1.2398472,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0,1.2398472,⋯,0,0.0000000,0.0000000,0.0000000,1.2398472,0.0000000,1.7766747,1.2398472,1.239847,0.0000000


In [10]:
exp

Row.names,LYZ,GAS7,MPO,ELANE,LPO,PRTN3,IDS,GBP5,PTCHD1,⋯,EBF1,GSDME,ARHGAP26,RAB31,ATP8B4,FNDC3B,GCNT1,ETV6,NSMCE2,seurat_clusters
<I<chr>>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
Flt3l_AAACGAAAGGAGACTC-1,0.000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0,⋯,0.0000000,0.0000000,1.8075560,0.0000000,1.8075560,0.0000000,0.0000000,1.807556,1.8075560,1
Flt3l_AAACGAAAGTCACGCC-1,0.000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0,⋯,1.0904147,0.0000000,1.0904147,0.0000000,0.0000000,0.0000000,0.0000000,1.599593,0.0000000,3
Flt3l_AAACGAACAAGACTAA-1,0.000000,0.6839030,0.0000000,0.0000000,1.0862677,0.0000000,0.0000000,0.0000000,0,⋯,0.0000000,0.6839030,1.5946060,0.0000000,0.6839030,0.6839030,0.0000000,1.086268,0.0000000,1
Flt3l_AAACGAACAAGCGAAC-1,0.000000,0.6852290,0.0000000,0.0000000,0.6852290,0.0000000,1.3743935,0.0000000,0,⋯,0.6852290,0.0000000,1.5967386,0.6852290,0.0000000,1.9322975,0.6852290,1.088041,0.0000000,5
Flt3l_AAACGAACACAGCCAC-1,0.000000,1.5160067,0.0000000,0.0000000,0.0000000,0.0000000,0.6357833,0.0000000,0,⋯,0.0000000,0.0000000,0.0000000,0.0000000,0.6357833,1.0213718,0.6357833,1.298965,1.2989654,9
Flt3l_AAACGAACACAGGTAG-1,0.000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0,⋯,0.0000000,0.0000000,0.0000000,0.0000000,1.5412375,0.0000000,0.0000000,0.000000,0.0000000,3
Flt3l_AAACGAACACAGTAGG-1,0.000000,0.6082509,0.9837364,0.0000000,0.0000000,0.0000000,0.6082509,0.0000000,0,⋯,0.0000000,0.6082509,1.6459773,0.9837364,1.2560848,1.2560848,0.6082509,1.925781,0.6082509,1
Flt3l_AAACGAACATAGAATG-1,1.394745,1.3947448,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0,⋯,0.0000000,0.0000000,0.0000000,1.3947448,1.3947448,0.0000000,1.3947448,1.394745,0.0000000,4
Flt3l_AAACGAACATGTAGAA-1,0.000000,1.2398472,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0,⋯,0.0000000,0.0000000,0.0000000,1.2398472,0.0000000,1.7766747,1.2398472,1.239847,0.0000000,1


In [11]:
outdir

[1] "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap"

In [16]:
pdf(file=file.path(outdir,"dotplot.pdf"), width=16, height=16)
DotPlot(object = se,features = markers, 
        group.by = "seurat_clusters") + 
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
#library(ggplot2)
dev.off()

Warning message in FetchData.Seurat(object = object, vars = features, cells = cells):
"The following requested variables were not found: D79A, MED13L "


png 
  2

In [17]:
pdf(file=file.path(outdir,"violin_markers.pdf"), width=16, height=16)
VlnPlot(se, features = markers, ncol=4)
dev.off()

Warning message in FetchData.Seurat(object = object, vars = features, slot = slot):
"The following requested variables were not found: D79A, MED13L "


png 
  2

In [14]:
pdf(file=file.path(outdir,"ridge.pdf"), width=32, height=32)
RidgePlot(se, features = markers, ncol=4)
dev.off()
#ggsave(file.path(outdir, "violin_markers.pdf"))

Warning message in FetchData.Seurat(object = object, vars = features, slot = slot):
"The following requested variables were not found: D79A, MED13L "
Picking joint bandwidth of 2.49e-06

Picking joint bandwidth of 0.116

Picking joint bandwidth of 0.00422

Picking joint bandwidth of 2.44e-06

Picking joint bandwidth of 0.0293

Picking joint bandwidth of 2.46e-06

Picking joint bandwidth of 0.0481

Picking joint bandwidth of 0.00766

Picking joint bandwidth of 0.011

Picking joint bandwidth of 0.0546

Picking joint bandwidth of 0.0376

Picking joint bandwidth of 0.0717

Picking joint bandwidth of 0.0344

Picking joint bandwidth of 0.0771

Picking joint bandwidth of 0.0166

Picking joint bandwidth of 2.41e-06

Picking joint bandwidth of 0.101

Picking joint bandwidth of 0.0378

Picking joint bandwidth of 0.0855

Picking joint bandwidth of 0.0959

Picking joint bandwidth of 0.00901

Picking joint bandwidth of 0.0127

Picking joint bandwidth of 2.4e-06

Picking joint bandwidth of 0.0272

P

png 
  2

## Plot marker embeddings

In [9]:
# featplot <- function(name.sig, se, curr.outdir, feat.names=NULL){
#   if (!is.null(feat.names)){
#     name <- feat.names[name.sig,]
#   }else{
#     name <- name.sig
#   }
#   if (name.sig %in% rownames(se)){
#     feat <- FeaturePlot(se,  features=name.sig) + ggtitle(name)
#     ggsave(plot=feat,
#            file=file.path(curr.outdir, paste0(name.sig,".embedFeat.top.png")))
#   }else{
#     print(paste0("Feature no in object: ", name.sig))
#   }

In [10]:
featplot <- function(name.sig, se, curr.outdir, use.density){
    if (name.sig %in% rownames(se)){
    feat <- FeaturePlot(se,  features=name.sig)
    ggsave(plot=feat,
           file=file.path(curr.outdir, paste0(name.sig,".embedFeat.top.png")))
    ggsave(plot=feat,
           file=file.path(curr.outdir, paste0(name.sig,".embedFeat.top.svg")))        
    if(use.density){
        print("plot density")
        f <- plot_density(se, name.sig) + ggtitle(name.sig)
        ggsave(plot=f,
           file=file.path(curr.outdir, paste0("density.", name.sig,".embedFeat.top.png")))
        ggsave(plot=f,
           file=file.path(curr.outdir, paste0("density.", name.sig,".embedFeat.top.svg")))
    }
    }
}


sapply(markers, FUN=featplot, se=se, curr.outdir=outdir, use.density=TRUE)

Saving 6.67 x 6.67 in image



[1] "plot density"


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



[1] "plot density"


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



...

[1] "plot density"


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



[1] "plot density"


Saving 6.67 x 6.67 in image



$CD34
[1] "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap/density.CD34.embedFeat.top.png"

$CD38
[1] "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap/density.CD38.embedFeat.top.png"

$PTPRC
[1] "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap/density.PTPRC.embedFeat.top.png"

$THY1
[1] "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap/density.THY1.embedFeat.top.png"

$ITGA6
[1] "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap/density.ITGA6.embedFeat.top.png"

$PF4
[1] "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap/density.PF4.embedFeat.top.png"

$CA1
[1] "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap/density.CA1.embedFeat.top.png"

$HBA2
[1] "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap/density.HBA2.embedFeat.top.png"

$LY86
[1] "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap/density.LY86.embedFeat.top.png"

$LYZ
[1] "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap/density.LYZ.embedFeat.top.png"

$GSTM1
[1] "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap/density.GSTM1.embedFeat.top.png"

$FCN1
[1] "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap/density.FCN1.embedFeat.top.png"

$LTF
[1] "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap/density.LTF.embedFeat.top.png"

$TPSAB1
[1] "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC1

## Plot marker embeddings with peaks

In [11]:
# DefaultAssay(se) <- "ATAC"
# feature.names <- ClosestFeature(se, rownames(se))


# featplot <- function(name.sig, se, curr.outdir, feat.names=NULL, use.density=TRUE){
#     if (!is.null(feat.names)){
#         name <- feat.names[name.sig,]
#     }else{
#         name <- name.sig
#     }
#     if (name.sig %in% rownames(se)){
#         feat <- FeaturePlot(se,  features=name.sig) + ggtitle(name)
#         ggsave(plot=feat,
#                file=file.path(curr.outdir, paste0(name.sig,".embedFeat.top.png")))  
#         if(use.density){
#             f <- plot_density(se, name.sig) + ggtitle(name)
#             ggsave(plot=f,
#                    file=file.path(curr.outdir, paste0("density.", name.sig,".embedFeat.top.png")))
#         }
#     }
# }

# marker.feature.names <- feature.names %>% dplyr::filter(gene_name %in% markers)
# rownames(marker.feature.names) <- marker.feature.names[,"query_region"]
# marker.feature.names

# marker.feature.names$name <- paste0(marker.feature.names$gene_name, "_", marker.feature.names$type)



# sapply(rownames(marker.feature.names), 
#        FUN=featplot, se=se, curr.outdir=outdir, feat.names=marker.feature.names["name"])

---

---

In [14]:
sessionInfo()

R version 4.1.1 (2021-08-10)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 20.04.4 LTS

Matrix products: default
BLAS/LAPACK: /home/isaac/miniconda3/envs/mttrace/lib/libopenblasp-r0.3.20.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] future_1.25.0             RColorBrewer_1.1-3       
 [3] metap_1.1                 cowplot_1.1.1            
 [5] magrittr_2.0.3            data.table_1.14.2        
 [7] EnsDb.Hsapiens.v75_2.99.0 ensembldb_2.18.1         
 [9] AnnotationFilter_1.18.0   GenomicFeatures_1.46.1   
[11] 